In [91]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# --- Step 1: Implement a Decision Tree (CART) ---
class DecisionTree:
    def __init__(self, max_depth=1, min_samples_split=2, regularization=0.01):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.regularization = regularization
        self.tree = None

    def fit(self, X, y, gradients, hessians, depth=0):
        m, n = X.shape

        # Stopping condition
        if depth >= self.max_depth or m < self.min_samples_split:
            return -np.sum(gradients) / (np.sum(hessians) + self.regularization)  # Newton-Raphson leaf value

        # Randomly select a subset of features
        num_features = max(2, int(np.sqrt(n)))
        selected_features = np.random.choice(n, num_features, replace=False)

        # Find best split
        best_feature, best_threshold, best_gain = None, None, -float("inf")
        for feature in selected_features:
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_mask = X[:, feature] <= threshold
                right_mask = X[:, feature] > threshold

                if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                    continue

                gain = ((np.sum(gradients[left_mask]) ** 2) / (np.sum(hessians[left_mask]) + self.regularization) +
                        (np.sum(gradients[right_mask]) ** 2) / (np.sum(hessians[right_mask]) + self.regularization) -
                        (np.sum(gradients) ** 2) / (np.sum(hessians) + self.regularization))

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold

        # Stop splitting if no gain
        if best_gain < self.regularization:
            return -np.sum(gradients) / (np.sum(hessians) + self.regularization)

        # Create tree structure
        left_mask = X[:, best_feature] <= best_threshold
        right_mask = X[:, best_feature] > best_threshold

        return {
            'feature_index': best_feature,
            'threshold': best_threshold,
            'left': self.fit(X[left_mask], y[left_mask], gradients[left_mask], hessians[left_mask], depth + 1),
            'right': self.fit(X[right_mask], y[right_mask], gradients[right_mask], hessians[right_mask], depth + 1)
        }

    def predict_sample(self, sample, node):
        if not isinstance(node, dict):
            return node
        if sample[node['feature_index']] <= node['threshold']:
            return self.predict_sample(sample, node['left'])
        else:
            return self.predict_sample(sample, node['right'])

    def predict(self, X):
        return np.array([self.predict_sample(sample, self.tree) for sample in X])


# --- Step 2: Implement XGBoost with Gradient Boosting & Log Loss ---
class XGBoostFromScratch:
    def __init__(self, n_estimators=200, learning_rate=0.1, max_depth=5, min_samples_split=10, regularization=0.01):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.regularization = regularization
        self.trees = []

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def fit(self, X, y):
        m = X.shape[0]
        prior_log_odds = np.log(np.mean(y) / (1 - np.mean(y)))
        predictions = np.full(m, prior_log_odds)

        for i in range(self.n_estimators):
            probabilities = np.clip(self.sigmoid(predictions), 1e-6, 1 - 1e-6)
            gradients = probabilities - y  # First derivative
            hessians = probabilities * (1 - probabilities)  # Second derivative

            tree = DecisionTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split, regularization=self.regularization)
            tree.tree = tree.fit(X, y, gradients, hessians)
            self.trees.append(tree)

            predictions += self.learning_rate * tree.predict(X)

    def predict(self, X):
        pred = np.zeros(X.shape[0])
        for tree in self.trees:
            pred += self.learning_rate * tree.predict(X)
        return (self.sigmoid(pred) >= 0.5).astype(int)


# --- Step 3: Load Titanic Dataset Locally and Train Model ---
if __name__ == "__main__":
    dataset_path = "../data/titanic.csv"  # Ensure this file exists
    df = pd.read_csv(dataset_path)
    df = df[["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]]
    df.dropna(inplace=True)
    df["Sex"] = df["Sex"].map({"male": 0, "female": 1})

    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()

    X = df.drop(columns=["Survived"]).values
    y = df["Survived"].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    model = XGBoostFromScratch(n_estimators=20, learning_rate=0.05, max_depth=8, min_samples_split=10, regularization=0.1)
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Model Accuracy: {accuracy:.4f}")

Model Accuracy: 0.7832
